# PDIoT Group A1 model

### Imports

In [1]:
import os
import numpy as np

# ml
import tensorflow as tf

# data
from ml.config import cfg
from ml.dataset import odgt2data
from ml.models import ModelBuilder, OptimizerBuilder, LRScheduleBuilder

# logging
from tqdm import tqdm

print("All packages imported!")


SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("Random seed set")

2023-11-09 12:01:08.504608: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


All packages imported!
Random seed set


### Config

In [2]:
config_fp = '/home/jonathan/git/pdiot-ml/cfg/log-motion.yaml'
cfg.merge_from_file(config_fp)
cfg.DATASET.path = '/home/jonathan/git/pdiot-ml/data/sets/pdiot-data/'
cfg.TRAIN.path = '/home/jonathan/git/pdiot-ml/ckpt/log-motion/'    

### Data

In [3]:
train_odgt_fp = os.path.join(cfg.DATASET.path, cfg.DATASET.LIST.train)
val_odgt_fp = os.path.join(cfg.DATASET.path, cfg.DATASET.LIST.val)

train_X, train_y = odgt2data(train_odgt_fp, cfg.MODEL.INPUT.window_size, cfg.TRAIN.DATA.overlap_size, cfg.DATASET.num_classes)
val_X, val_y = odgt2data(val_odgt_fp, cfg.MODEL.INPUT.window_size, cfg.TRAIN.DATA.overlap_size, cfg.DATASET.num_classes)

### Model

In [4]:
model = ModelBuilder.build_classifier(cfg.MODEL, '', num_classes=2)
optimizer = OptimizerBuilder.build_optimizer(cfg.TRAIN.OPTIM)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

lr_scheduler = LRScheduleBuilder.build_scheduler(cfg.TRAIN.LR)
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=cfg.TRAIN.LEN.early_stop)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model.fit(train_X, train_y, validation_data=(val_X, val_y), 
          epochs=cfg.TRAIN.LEN.num_epoch, 
          batch_size=cfg.TRAIN.DATA.batch_size, 
          initial_epoch=cfg.TRAIN.LEN.start_epoch,
          workers=cfg.TRAIN.DATA.num_workers,
          callbacks=[lr_callback, 
                     early_stop_callback])

Epoch 1/100


2023-11-09 12:01:13.847342: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-09 12:01:13.849056: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
/home/jonathan/miniconda3/envs/pdiot/lib/python3.10/site-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


177/177 [==============================] - 2s 7ms/step - loss: 2.0683 - accuracy: 0.5995 - val_loss: 2.0035 - val_accuracy: 0.5166 - lr: 1.0000e-04
Epoch 2/100
177/177 [==============================] - 1s 6ms/step - loss: 1.5423 - accuracy: 0.6498 - val_loss: 1.6919 - val_accuracy: 0.5762 - lr: 1.0000e-04
Epoch 3/100
177/177 [==============================] - 1s 6ms/step - loss: 1.2527 - accuracy: 0.6841 - val_loss: 1.4056 - val_accuracy: 0.6225 - lr: 1.0000e-04
Epoch 4/100
177/177 [==============================] - 1s 7ms/step - loss: 1.0586 - accuracy: 0.7115 - val_loss: 1.2035 - val_accuracy: 0.6689 - lr: 1.0000e-04
Epoch 5/100
177/177 [==============================] - 1s 5ms/step - loss: 0.9278 - accuracy: 0.7302 - val_loss: 1.0444 - val_accuracy: 0.7086 - lr: 1.0000e-04
Epoch 6/100
177/177 [==============================] - 1s 6ms/step - loss: 0.8323 - accuracy: 0.7420 - val_loss: 0.9541 - val_accuracy: 0.7219 - lr: 1.0000e-04
Epoch 7/100
177/177 [==============================]